# Book Problem


Cierta compañía editorial produce libros y revistas especializados, los cuales vende a $30000 y $25000 por unidad, respectivamente. Se ha estimado que hay una disponibilidad de 300 horas en revisión técnica, 350 horas en impresión y 400 horas en empaste mensuales. Establezca la cantidad de libros y revistas que se deben producir por mes si se sabe que para producir un libro se requieren 6h de revisión técnica, 5h en impresión y 10h en empaste, mientras que para producir una revista se requieren 5h en revisión técnica, 7h en impresión y 4h en empaste



| Product | Technical review | Printing | Binding | Price |
| ------- | ---------------- | -------- | ------- | ----- |
| Book   | 6 hours  | 5 hours | 10 hours | $30000 |
| Magazine | 5 hours | 7 hours | 4 hours | $25000 | 
| Total maximum | 300 hours | 350 hours | 400 hours| 

# Problem statement

## Sets
 
$P$ = Set of Products

$O$ = Set of Product Operations

## Decision Variables

$x_{p}$   -> Number of products $p$ produced 

## Parameters

$V_{p}$ -> price of products, $\forall p \in P$

$T_{p,o}$ -> Time required to complete an operation $o$ per product $p$, $\forall o \in O$, $\forall p \in P$

$Timax_{o}$ -> Maximun time spent monthtly in operation $o$, $\forall o \in O$

## Constricts

1. Maximun time spent:

$$\sum_{\forall p \in P} T_{p,o} x_{p} \leq Timax_{o}, \forall o \in O$$

2. Basic constrain:

$$x_{p} \geq 0$$

$$x_{p} \in Z$$


## Objective

$$\text{max} \sum_{\forall p \in P} V_{p} x_{p}$$

# Model

In [28]:
import pyomo.environ as pyo

In [29]:
model = pyo.ConcreteModel(name="Book Problem", doc="Book Problem")

In [30]:
# Define sets
PRODUCTS = ["Book", "Magazine"]
OPERATIONS = ["Technical Review", "Printing", "Binding"]

model.P =pyo.Set(initialize=PRODUCTS, doc="Product Set")
model.O = pyo.Set(initialize=OPERATIONS, doc="Operations Set")

In [31]:
#Define Parameters
vp = {"Book": 30000, 
    "Magazine": 25000}

tpo = {
    ("Book", "Technical Review"): 6,
    ("Book", "Printing"): 5,
    ("Book", "Binding"): 10,
    ("Magazine", "Technical Review"): 5,
    ("Magazine", "Printing"): 7,
    ("Magazine", "Binding"): 4
}

timaxo = {
    "Technical Review": 300,
    "Printing": 350,
    "Binding": 400
}

model.vp = pyo.Param(model.P, initialize=vp, doc="Price per Product")
model.tpo = pyo.Param(model.P, model.O, initialize=tpo, doc="Time of Operation per Product")
model.timaxo = pyo.Param(model.O, initialize=timaxo, doc="Maximum Time per Operation")

In [32]:
# Define Variables
model.xp = pyo.Var(model.P, domain=pyo.NonNegativeIntegers, doc="Amount of products to produce")

In [33]:
# Define Constraints
#1. Maximum time constraint:
def time_limit_max(model, o):
    return sum(model.tpo[p,o] * model.xp[p] for p in model.P) <= model.timaxo[o]

model.time_limit_max = pyo.Constraint(model.O, rule=time_limit_max, doc="Maximum Time per Operation")


In [34]:
# Define Objective
def cost_function(model):
    return sum(model.vp[p] * model.xp[p] for p in model.P)

model.objective = pyo.Objective(rule=cost_function, sense=pyo.maximize, doc="Total Profit of the products produced")

In [35]:
model.pprint()

Book Problem

    3 Set Declarations
        O : Operations Set
            Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    3 : {'Technical Review', 'Printing', 'Binding'}
        P : Product Set
            Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    2 : {'Book', 'Magazine'}
        tpo_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain : Size : Members
            None :     2 :    P*O :    6 : {('Book', 'Technical Review'), ('Book', 'Printing'), ('Book', 'Binding'), ('Magazine', 'Technical Review'), ('Magazine', 'Printing'), ('Magazine', 'Binding')}

    3 Param Declarations
        timaxo : Maximum Time per Operation
            Size=3, Index=O, Domain=Any, Default=None, Mutable=False
            Key              : Value
                     Binding :   400
                    Printing :   350
   

## Solve the model

In [36]:
solver = pyo.SolverFactory('appsi_highs')

In [37]:
solver.solve(model, tee=True)

Presolving model
3 rows, 2 cols, 6 nonzeros
3 rows, 2 cols, 6 nonzeros
Objective function is integral with scale 0.0002

Solving MIP model with:
   3 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   6 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   2450000         -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   2450000         1475000           66.10%        0      0      0         0     0.0s
         0       0         0   0.00%   1500000         1475000            1.69%        0      0      1         3     0.0s
 H       0       0         0   0.00%   1500000         1500000            0.00%        3      1      2         4     0.0s

Solving report
  Status           

{'Problem': [{'Lower bound': 1500000.0, 'Upper bound': 1500000.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': -1}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [38]:
model.xp.display()

xp : Amount of products to produce
    Size=2, Index=P
    Key      : Lower : Value              : Upper : Fixed : Stale : Domain
        Book :     0 : 30.000000000000007 :  None : False : False : NonNegativeIntegers
    Magazine :     0 :  23.99999999999999 :  None : False : False : NonNegativeIntegers


In [39]:
model.objective.expr()

1500000.0